# To do:

Structure data so that yelp and foursquare IDs are matched to menus

Combine foursquare and yelp menus

Get prices from Foursquare

In [1]:
import requests
import os
import importlib
import pandas as pd
import numpy as np
%matplotlib inline
import pdb
import json
import time
from Kibbeh.src import feature_extract_resto
from Kibbeh.src import API_io
import re
import matplotlib.pyplot as plt
import ast

In [2]:
#working_dir = 'C:\\Users\\Chauncey\\Documents\\GitHub\\Kibbeh'
working_dir = 'C:\\Users\\Me\\Documents\\GitHub\\Kibbeh'
seattle_zips = pd.read_csv(working_dir + '\\seattle_zips.csv')['Zip'].values

In [3]:
# load foursquare restaurants and ratings
with open('fs_restaurant_list.json', 'r') as json_file:
    fs_top_restos = json.load( json_file)
len(fs_top_restos)

3081

In [4]:
# load foursquare menus
with open('fs_restaurant_menus.json', 'r') as json_file:
    fs_menus_rated = json.load( json_file)
len(fs_menus_rated)

332

In [5]:
# load yelp restaurants
with open('yelp_restaurant_list.json', 'r') as json_file:
    yelp_restos = json.load( json_file)
len(yelp_restos)

1000

In [6]:
# load from local json
with open('locu_restaurant_list.json', 'r') as json_file:
    locu_restos = json.load( json_file)
len(locu_restos)

1564

In [7]:
# load
with open('locu_menu_list.json', 'r') as json_file:
    locu_menus = json.load( json_file)
len(locu_menus)

399

# Scrape Yelp data

In [ ]:
from yelpapi import YelpAPI
with open(working_dir+ '\\yelp_api_key.txt', 'r') as api_file:
    api_key =  api_file.read()
    [CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET] = api_key.split('\n')
yelp_api = YelpAPI(CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET)

In [ ]:
yelp_restos = API_io.get_yelp_restaurants(yelp_api)
len(yelp_restos)

In [ ]:
# save
with open('yelp_restaurant_list.json', 'w') as json_file:
    json.dump(yelp_restos, json_file)

# Scrape Foursquare data
### Top restaurants in each zip code

In [ ]:
import foursquare
with open(working_dir+ '\\foursquare_api_key.txt', 'r') as api_file:
    api_key =  api_file.read()
    [FOURSQUARE_ID, FOURSQUARE_SECRET] = api_key.split('\n')
foursquare_client = foursquare.Foursquare(FOURSQUARE_ID, FOURSQUARE_SECRET)

In [ ]:
# get data from all seattle zip codes
fs_top_restos = API_io.get_fs_top_restos_zip(foursquare_client, seattle_zips[:2])
# this takes a while to run
len(fs_top_restos)

In [ ]:
fs_top_restos = remove_duplicate_fs_restos(fs_top_restos)
len(fs_top_restos)

In [ ]:
# save
with open('fs_restaurant_list.json', 'w') as json_file:
    json.dump(fs_top_restos, json_file)

### Random restaurants in a latitude longitude grid
Most of these do not have ratings or menus

In [ ]:
# not tested after offloading to API_io.py
seattle_grid = [-122.38, -122.28, 47.55, 47.70]
fs_ll_restos = API_io.get_fs_restos_by_ll(seattle_grid, foursquare_client, grid_step = 0.1)

In [ ]:
len(fs_ll_restos)

### Get menus from foursquare

In [ ]:
# get foursquare venues that have both menu and rating
# use set intersection to get intersection of keys
fs_has_menu_rated = [x['venue'] for x in fs_top_restos if set(('menu', 'rating')) <= x['venue'].keys()]
len(fs_has_menu_rated)

In [ ]:
fs_menus_rated = API_io.get_menus_fs(foursquare_client,fs_id_list[:10])
len(fs_menus_rated) # there are less menus than you hope because some menus not availabe through API

In [ ]:
# save menus
with open('fs_restaurant_menus.json', 'w') as json_file:
    json.dump(fs_menus_rated, json_file)

# Load OpenMenu data
Denied on OpenMenu API request

# Load Locu data
### Set up api key and class

In [ ]:
from Kibbeh.src.locu_api import VenueApiClient, MenuItemApiClient
with open(working_dir+ '\\locu_api_key.txt', 'r') as api_file:
    api_key =  api_file.read()
    [LOCU_KEY] = api_key.split('\n')
locu_venue_client = VenueApiClient(LOCU_KEY)

### Get list of locu restaurants by zip code

In [ ]:
import itertools
def get_locu_by_zip(venue_client, zip_list):
    # a set of 25 restaurants with menus for each zip code in seattle
    list_of_lists =  list(map(lambda x: venue_client.search(category=['restaurant'], has_menu=True, postal_code=x)['objects'],
                              zip_list) )
    return list(itertools.chain(*list_of_lists))

#venues = venue_client.search(locality = 'Seattle', has_menu=True)

In [ ]:
locu_restos = get_locu_by_zip(venue_client, seattle_zips)
len(locu_restos)

In [ ]:
# save
with open('locu_restaurant_list.json', 'w') as json_file:
    json.dump(locu_restos, json_file)